# Mixing Secrets test set preparation

This notebook prepares the 'real' test set based on the Mixing Secrets collection. Run `../download.ipynb` first.

Note: As of version 2.1b6.dev234, the Essentia library has a [bug](https://github.com/MTG/essentia/issues/1054) that causes an infinite loop for some inputs.
To avoid this, you have to build our patched version of Essentia: https://github.com/cifkao/essentia/tree/patched

Copyright 2020 InterDigital R&D and Télécom Paris.  
Author: Ondřej Cífka

In [1]:
import collections
import concurrent.futures as cf
import glob
import hashlib
import json
import os
import regex as re
import shutil
import sys
import traceback

import essentia
import essentia.standard as estd
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pretty_midi
import soundfile as sf
from tqdm.auto import tqdm

In [2]:
INPUT_DIR = '../download'
OUTPUT_DIR = 'wav_16kHz'
SR_IN = 16000
SR = 16000
MAX_FILES_PER_SONG = 5
MAX_SEGMENTS_PER_FILE = 4
MAX_SEGMENTS_PER_INSTRUMENT = 5

In [3]:
TRACK_BLACKLIST = [
    'kick', 'snare', 'overhead', 'tom', 'drum', 'hat', 'cymbal', 'sharedown', 'ride', 'crash',
    'cowbell', 'stick', 'clap', 'gong', 'shaker', 'djembe', 'cabasa', 'cajon', 'bongo', 'bonga', 'conga',
    'timp', 'tambourine', 'bombo', 'timbal', 'woodblock', 'triangle', 'rimshot',
    'percussion', 'noise', 'vocal', 'vox', 'sfx', 'loop', 'room', 'spot', 'mix'
]
TRACK_BLACKLIST_RE = re.compile('(' + '|'.join(x for x in TRACK_BLACKLIST) + ')')

In [4]:
def normalize_instrument_name(name):
    name = re.sub(r'[^\p{L}]', ' ', name)
    name = re.sub(r'(\p{Ll})(\p{Lu})', r'\1 \2', name)
    name = re.sub(r'\s+', ' ', name)
    name = name.strip().lower()
    return name

In [5]:
audio_paths = []
excluded_names = set()
included_names = set()

for song_path in glob.glob(os.path.join(INPUT_DIR, '*')):
    # Use song name as seed
    seed = os.path.basename(song_path).encode()
    seed = int.from_bytes(hashlib.sha512(seed).digest(), 'big')
    rng = np.random.default_rng(seed=seed)

    song_audio_paths = []
    for path in glob.glob(os.path.join(song_path, '**', '*.*'), recursive=True):
        if not os.path.splitext(path)[1] in ['.wav', '.flac']:
            continue
        name, _ = os.path.splitext(os.path.basename(path))
        name = normalize_instrument_name(name)
        if TRACK_BLACKLIST_RE.search(name) or len(name) == 0:
            excluded_names.add(name)
            continue
        included_names.add(name)
        song_audio_paths.append(path)
    
    song_audio_paths.sort()
    rng.shuffle(song_audio_paths)
    audio_paths.extend(song_audio_paths[:MAX_FILES_PER_SONG])
audio_paths.sort()

In [6]:
def get_pitches(audio, sr):
    input_sr, sr = sr, 8000  # Need to resample because of EqualLoudness
    audio = estd.Resample(inputSampleRate=input_sr, outputSampleRate=sr)(audio)
    audio = estd.EqualLoudness(sampleRate=sr)(audio)
    rng = np.random.default_rng(seed=(audio > 0).sum())
    audio = rng.normal(loc=audio, scale=1e-4).astype(audio.dtype)  # To prevent Melodia from crashing
    pitches = estd.MultiPitchMelodia(sampleRate=sr)(audio)
    pitches = [[pretty_midi.utilities.hz_to_note_number(p) for p in pl if not np.isclose(0, p)]
               for pl in pitches]
    pitches = [[int(p + 0.5) for p in pl] for pl in pitches]
    return pitches

In [7]:
def process_file(path):
    # Use filename as seed
    seed = os.path.basename(path).encode()
    seed = int.from_bytes(hashlib.sha512(seed).digest(), 'big')
    rng = np.random.default_rng(seed=seed)
    
    instrument_name = normalize_instrument_name(os.path.splitext(os.path.basename(path))[0])

    audio_full = estd.EasyLoader(filename=path, sampleRate=SR_IN)()
    frame_size = SR_IN // 10
    silence_alg = estd.SilenceRate(thresholds=[essentia.db2lin(-60 / 2)])
    silence = np.array([silence_alg(frame) for frame in 
                        estd.FrameGenerator(audio_full, frameSize=frame_size, hopSize=frame_size)]).reshape(-1)
    
    # Find 8-second segments with < 50% silence and starting with a non-silent frame
    seg_len = 8 * SR_IN // frame_size
    silence_cumsum = np.pad(np.cumsum(silence), (1, 0))
    silence_sums = silence_cumsum[seg_len + 1:-1] - silence_cumsum[1:-seg_len - 1]
    [candidates] = np.where((silence_sums < seg_len // 2) & (silence[:-seg_len - 1] == 0))
    
    # Prevent overlapping segments by allowing only one every 8 seconds
    candidates_filtered = []
    for i in candidates:
        if not candidates_filtered or candidates_filtered[-1] <= i - seg_len:
            candidates_filtered.append(i)
    candidates = candidates_filtered
    
    if len(candidates) == 0:
        return []
    
    metadata = []
    rng.shuffle(candidates)
    for i in candidates[:MAX_SEGMENTS_PER_FILE]:
        i *= frame_size
        audio = audio_full[i:i + 8 * SR_IN]

        pitches = get_pitches(audio, sr=SR_IN)
        if len(pitches) == 0 or all(len(pl) == 0 for pl in pitches):
            continue
        num_voices = np.mean([len(pl) for pl in pitches if len(pl) > 0])
        mean_pitch = np.mean([p for pl in pitches for p in pl])

        out_path = os.path.join(OUTPUT_DIR,
                                os.path.relpath(path, INPUT_DIR).replace(os.path.sep, '_'))
        out_path = '{}.{}.wav'.format(os.path.splitext(out_path)[0], str(i // SR_IN).zfill(len(str(candidates[-1]))))
        sf.write(out_path, estd.Resample(inputSampleRate=SR_IN, outputSampleRate=SR)(audio), samplerate=SR)

        metadata.append({
            'path': os.path.relpath(out_path, OUTPUT_DIR),
            'track_name': os.path.relpath(path, INPUT_DIR).split(os.path.sep)[0],
            'instrument': instrument_name,
            'src_path': os.path.relpath(path, INPUT_DIR),
            'src_offset': i / SR_IN,
            'stats': {
                'num_voices': num_voices,
                'mean_pitch': mean_pitch
            }
        })
        print(metadata[-1])
    return metadata

In [9]:
os.makedirs(OUTPUT_DIR)
with cf.ProcessPoolExecutor(32) as pool:
    results = [item
               for items in tqdm(pool.map(process_file, audio_paths), total=len(audio_paths))
               for item in items]

{'path': 'APZX_Transcention_APZX_Transcention_Full_14_SynthArp1.026.wav', 'track_name': 'APZX_Transcention', 'instrument': 'synth arp', 'src_path': 'APZX_Transcention/APZX_Transcention_Full/14_SynthArp1.wav', 'src_offset': 26.4, 'stats': {'num_voices': 1.010230179028133, 'mean_pitch': 77.85822784810127}}
{'path': 'AbletonesBigBand_CorineCorine_PIANO B - ELA M 260 - Neve 33102.0088.wav', 'track_name': 'AbletonesBigBand_CorineCorine', 'instrument': 'piano b ela m neve', 'src_path': 'AbletonesBigBand_CorineCorine/PIANO B - ELA M 260 - Neve 33102.wav', 'src_offset': 88.8, 'stats': {'num_voices': 1.2564102564102564, 'mean_pitch': 57.48979591836735}}
{'path': 'AbletonesBigBand_CorineCorine_E GUITAR - AR-51 - Presonus M80.0001.wav', 'track_name': 'AbletonesBigBand_CorineCorine', 'instrument': 'e guitar ar presonus m', 'src_path': 'AbletonesBigBand_CorineCorine/E GUITAR - AR-51 - Presonus M80.wav', 'src_offset': 1.4, 'stats': {'num_voices': 1.312785388127854, 'mean_pitch': 56.420869565217394}}

{'path': 'APZX_CyberMower_APZX_CyberMower_Full_21_Synth03.0275.wav', 'track_name': 'APZX_CyberMower', 'instrument': 'synth', 'src_path': 'APZX_CyberMower/APZX_CyberMower_Full/21_Synth03.wav', 'src_offset': 275.1, 'stats': {'num_voices': 1.0, 'mean_pitch': 81.00925925925925}}
{'path': 'APZX_CyberMower_APZX_CyberMower_Full_24_Synth06.0268.wav', 'track_name': 'APZX_CyberMower', 'instrument': 'synth', 'src_path': 'APZX_CyberMower/APZX_CyberMower_Full/24_Synth06.wav', 'src_offset': 268.0, 'stats': {'num_voices': 1.5, 'mean_pitch': 73.71969696969697}}{'path': 'APZX_CyberMower_APZX_CyberMower_Full_22_Synth04.0089.wav', 'track_name': 'APZX_CyberMower', 'instrument': 'synth', 'src_path': 'APZX_CyberMower/APZX_CyberMower_Full/22_Synth04.wav', 'src_offset': 89.8, 'stats': {'num_voices': 1.002267573696145, 'mean_pitch': 71.47737556561086}}

{'path': 'APZX_CyberMower_APZX_CyberMower_Full_31_Synth13.0259.wav', 'track_name': 'APZX_CyberMower', 'instrument': 'synth', 'src_path': 'APZX_CyberMower/APZX_

{'path': 'AbletonesBigBand_SongOfIndia_UPRIGHT BASS - ELA M 260 - Neve 33102.104.wav', 'track_name': 'AbletonesBigBand_SongOfIndia', 'instrument': 'upright bass ela m neve', 'src_path': 'AbletonesBigBand_SongOfIndia/UPRIGHT BASS - ELA M 260 - Neve 33102.wav', 'src_offset': 104.8, 'stats': {'num_voices': 1.0144404332129964, 'mean_pitch': 50.188612099644125}}{'path': 'APZX_Amalgamize_APZX_Amalgamize_Full_08_Synth3.0289.wav', 'track_name': 'APZX_Amalgamize', 'instrument': 'synth', 'src_path': 'APZX_Amalgamize/APZX_Amalgamize_Full/08_Synth3.wav', 'src_offset': 289.3, 'stats': {'num_voices': 1.0701754385964912, 'mean_pitch': 65.01639344262296}}

{'path': 'APZX_Lacuna_APZX_Lacuna_Full_27_Synth04.0175.wav', 'track_name': 'APZX_Lacuna', 'instrument': 'synth', 'src_path': 'APZX_Lacuna/APZX_Lacuna_Full/27_Synth04.wav', 'src_offset': 175.2, 'stats': {'num_voices': 1.1071428571428572, 'mean_pitch': 61.225806451612904}}
{'path': 'Actions_OneMinuteSmile_Actions_OneMinuteSmile_Full_11_ElecGtr4.019.wa

{'path': 'AdamBuckley_DragMeDown_AdamBuckley_DragMeDown_Full_34_ElecGtr5.0099.wav', 'track_name': 'AdamBuckley_DragMeDown', 'instrument': 'elec gtr', 'src_path': 'AdamBuckley_DragMeDown/AdamBuckley_DragMeDown_Full/34_ElecGtr5.wav', 'src_offset': 99.4, 'stats': {'num_voices': 1.1258134490238612, 'mean_pitch': 56.70712909441233}}
{'path': 'Actions_OneMinuteSmile_Actions_OneMinuteSmile_Full_12_ElecGtr5.0139.wav', 'track_name': 'Actions_OneMinuteSmile', 'instrument': 'elec gtr', 'src_path': 'Actions_OneMinuteSmile/Actions_OneMinuteSmile_Full/12_ElecGtr5.wav', 'src_offset': 139.6, 'stats': {'num_voices': 1.0439814814814814, 'mean_pitch': 68.63858093126386}}
{'path': 'Actions_SouthOfTheWater_Actions_SouthOfTheWater_Full_14_ElecGtr5.0099.wav', 'track_name': 'Actions_SouthOfTheWater', 'instrument': 'elec gtr', 'src_path': 'Actions_SouthOfTheWater/Actions_SouthOfTheWater_Full/14_ElecGtr5.wav', 'src_offset': 99.4, 'stats': {'num_voices': 1.19375, 'mean_pitch': 52.92931937172775}}
{'path': 'Actio

{'path': 'AleLak_NossoMundoDeixouDeExistir_AleLak_NossoMundoDeixouDeExistir_Full_16_ElecGtr03Amp1.126.wav', 'track_name': 'AleLak_NossoMundoDeixouDeExistir', 'instrument': 'elec gtr amp', 'src_path': 'AleLak_NossoMundoDeixouDeExistir/AleLak_NossoMundoDeixouDeExistir_Full/16_ElecGtr03Amp1.wav', 'src_offset': 126.6, 'stats': {'num_voices': 1.5356125356125356, 'mean_pitch': 72.00742115027829}}
{'path': 'AjayNagarkotiAndVibhuDixit_AeMereHumsafar_AjayNagarkotiAndVibhuDixit_AeMereHumsafar_Full_10_AcousticGtr2.0055.wav', 'track_name': 'AjayNagarkotiAndVibhuDixit_AeMereHumsafar', 'instrument': 'acoustic gtr', 'src_path': 'AjayNagarkotiAndVibhuDixit_AeMereHumsafar/AjayNagarkotiAndVibhuDixit_AeMereHumsafar_Full/10_AcousticGtr2.wav', 'src_offset': 55.0, 'stats': {'num_voices': 1.198051948051948, 'mean_pitch': 54.823848238482384}}
{'path': 'AjayNagarkotiAndVibhuDixit_AeMereHumsafar_AjayNagarkotiAndVibhuDixit_AeMereHumsafar_Full_12_Piano1.0050.wav', 'track_name': 'AjayNagarkotiAndVibhuDixit_AeMereH

...

{'path': 'Woodfire_Outer_Woodfire_Outer_Full_08_BassDI.0027.wav', 'track_name': 'Woodfire_Outer', 'instrument': 'bass di', 'src_path': 'Woodfire_Outer/Woodfire_Outer_Full/08_BassDI.wav', 'src_offset': 27.4, 'stats': {'num_voices': 1.1401869158878504, 'mean_pitch': 40.85655737704918}}{'path': 'Woodfire_WealthyInTime_Woodfire_WealthyInTime_Full_11_ElecGtr02Mic1.0111.wav', 'track_name': 'Woodfire_WealthyInTime', 'instrument': 'elec gtr mic', 'src_path': 'Woodfire_WealthyInTime/Woodfire_WealthyInTime_Full/11_ElecGtr02Mic1.wav', 'src_offset': 111.6, 'stats': {'num_voices': 1.0, 'mean_pitch': 66.14651162790697}}{'path': 'Woodfire_Outer_Woodfire_Outer_Full_13_ElecGtr04Mic1.0200.wav', 'track_name': 'Woodfire_Outer', 'instrument': 'elec gtr mic', 'src_path': 'Woodfire_Outer/Woodfire_Outer_Full/13_ElecGtr04Mic1.wav', 'src_offset': 200.2, 'stats': {'num_voices': 1.7377521613832854, 'mean_pitch': 69.23383084577114}}


{'path': 'Woodfire_Outer_Woodfire_Outer_Full_08_BassDI.0203.wav', 'track_name': 

{'path': 'WovenHollow_TellMeNice_WovenHollow_TellMeNice_Full_08_BassDI.0160.wav', 'track_name': 'WovenHollow_TellMeNice', 'instrument': 'bass di', 'src_path': 'WovenHollow_TellMeNice/WovenHollow_TellMeNice_Full/08_BassDI.wav', 'src_offset': 160.5, 'stats': {'num_voices': 1.0371352785145889, 'mean_pitch': 40.73913043478261}}{'path': 'ZaneCarneyAndFriends_TalkToMeBaby_Zane Carney @ Vintage King LA TELEFUNKEN_E GUITAR 3 - M81 - API 1608.0114.wav', 'track_name': 'ZaneCarneyAndFriends_TalkToMeBaby', 'instrument': 'e guitar m api', 'src_path': 'ZaneCarneyAndFriends_TalkToMeBaby/Zane Carney @ Vintage King LA TELEFUNKEN/E GUITAR 3 - M81 - API 1608.wav', 'src_offset': 114.7, 'stats': {'num_voices': 1.1688311688311688, 'mean_pitch': 59.80833333333333}}

{'path': 'YoungGriffo_BloodToBone_YoungGriffo_BloodToBone_Full_14_Gtr2Amp1.0112.wav', 'track_name': 'YoungGriffo_BloodToBone', 'instrument': 'gtr amp', 'src_path': 'YoungGriffo_BloodToBone/YoungGriffo_BloodToBone_Full/14_Gtr2Amp1.wav', 'src_offse

{'path': 'Zwiepack_Air_Zwiepack_Air_Full_05_SaxMic2_PeerArray.032.wav', 'track_name': 'Zwiepack_Air', 'instrument': 'sax mic peer array', 'src_path': 'Zwiepack_Air/Zwiepack_Air_Full/05_SaxMic2_PeerArray.wav', 'src_offset': 32.1, 'stats': {'num_voices': 1.0, 'mean_pitch': 71.97968397291196}}
{'path': 'Zwiepack_Air_Zwiepack_Air_Full_03_BassMic2_PeerArray.115.wav', 'track_name': 'Zwiepack_Air', 'instrument': 'bass mic peer array', 'src_path': 'Zwiepack_Air/Zwiepack_Air_Full/03_BassMic2_PeerArray.wav', 'src_offset': 115.0, 'stats': {'num_voices': 1.0, 'mean_pitch': 72.2837528604119}}
{'path': 'Zwiepack_Air_Zwiepack_Air_Full_05_SaxMic2_PeerArray.162.wav', 'track_name': 'Zwiepack_Air', 'instrument': 'sax mic peer array', 'src_path': 'Zwiepack_Air/Zwiepack_Air_Full/05_SaxMic2_PeerArray.wav', 'src_offset': 162.1, 'stats': {'num_voices': 1.0, 'mean_pitch': 72.67328918322296}}
{'path': 'Zwiepack_Air_Zwiepack_Air_Full_05_SaxMic2_PeerArray.040.wav', 'track_name': 'Zwiepack_Air', 'instrument': 'sax

{'path': 'cryonicPAX_Excessive_cryonicPAX_Excessive_Full_13_Bass.0245.wav', 'track_name': 'cryonicPAX_Excessive', 'instrument': 'bass', 'src_path': 'cryonicPAX_Excessive/cryonicPAX_Excessive_Full/13_Bass.wav', 'src_offset': 245.7, 'stats': {'num_voices': 2.1696606786427144, 'mean_pitch': 62.945722171113154}}
{'path': 'thelvnguage_KingsAndQueens_thelvnguage_KingsAndQueens_Full_05_BassAmp.020.wav', 'track_name': 'thelvnguage_KingsAndQueens', 'instrument': 'bass amp', 'src_path': 'thelvnguage_KingsAndQueens/thelvnguage_KingsAndQueens_Full/05_BassAmp.wav', 'src_offset': 20.9, 'stats': {'num_voices': 1.305050505050505, 'mean_pitch': 41.79102167182663}}{'path': 'cryonicPAX_Excessive_cryonicPAX_Excessive_Full_15_Synth2.0021.wav', 'track_name': 'cryonicPAX_Excessive', 'instrument': 'synth', 'src_path': 'cryonicPAX_Excessive/cryonicPAX_Excessive_Full/15_Synth2.wav', 'src_offset': 21.7, 'stats': {'num_voices': 1.2278761061946903, 'mean_pitch': 67.82522522522522}}

{'path': 'cryonicPAX_HoldMe_cry

{'path': 'cryonicPAX_Melancholy_cryonicPAX_Melancholy_Full_10_Bells.0104.wav', 'track_name': 'cryonicPAX_Melancholy', 'instrument': 'bells', 'src_path': 'cryonicPAX_Melancholy/cryonicPAX_Melancholy_Full/10_Bells.wav', 'src_offset': 104.0, 'stats': {'num_voices': 1.1891891891891893, 'mean_pitch': 58.99318181818182}}
{'path': 'cfx_Mathematician_cfx_Mathematician_Full_11_Synth1Dry.0139.wav', 'track_name': 'cfx_Mathematician', 'instrument': 'synth dry', 'src_path': 'cfx_Mathematician/cfx_Mathematician_Full/11_Synth1Dry.wav', 'src_offset': 139.4, 'stats': {'num_voices': 1.0824053452115814, 'mean_pitch': 43.95679012345679}}
{'path': 'cfx_Mathematician_cfx_Mathematician_Full_12_Synth1Wet.319.wav', 'track_name': 'cfx_Mathematician', 'instrument': 'synth wet', 'src_path': 'cfx_Mathematician/cfx_Mathematician_Full/12_Synth1Wet.wav', 'src_offset': 319.8, 'stats': {'num_voices': 1.1985111662531018, 'mean_pitch': 50.56107660455486}}
{'path': 'cryonicPAX_Melancholy_cryonicPAX_Melancholy_Full_10_Bell

In [10]:
results_by_instrument = collections.defaultdict(list)
for item in results:
    results_by_instrument[item['instrument']].append(item)

sorted((len(v), k) for k, v in results_by_instrument.items())

[(1, 'bgv'),
 (1, 'brass'),
 (1, 'cello samples'),
 (1, 'glock'),
 (1, 'gtr a mic'),
 (1, 'horns bone ela m'),
 (1, 'horns trumpet m'),
 (1, 'keys toy piano ela m'),
 (1, 'keys yamaha di'),
 (1, 'nord r di'),
 (1, 'pizz'),
 (1, 'sample brass'),
 (1, 'sax od'),
 (1, 'speech'),
 (1, 'stylophone'),
 (1, 'traffic'),
 (2, 'adlib group omni u ssl e'),
 (2, 'banjo'),
 (2, 'cello bridge'),
 (2, 'cello harm hi mic'),
 (2, 'cellos'),
 (2, 'ethno flute'),
 (2, 'guitalele'),
 (2, 'hall pair'),
 (2, 'midigtr'),
 (2, 'outro alto e ssl e'),
 (2, 'pizz bass'),
 (2, 'synth reverse'),
 (2, 'trombone b ar neve'),
 (2, 'trombone m api'),
 (2, 'violin solo'),
 (3, 'chamber m'),
 (3, 'clarinet'),
 (3, 'decca tree right'),
 (3, 'fiddles mic'),
 (3, 'guitar condenser tory cu'),
 (3, 'keys melodica c'),
 (3, 'keys piano m'),
 (3, 'keys whirly amp m'),
 (3, 'mandolin ela m'),
 (3, 'nord l di'),
 (3, 'piano inside mics'),
 (3, 'sax alto'),
 (3, 'sax tenor'),
 (3, 'soprano'),
 (3, 'strings section mic vln i'),
 (

In [11]:
rng = np.random.default_rng(0)
num_total = len(results)
num_removed = 0
for _, result_list in sorted(results_by_instrument.items()):
    result_list.sort(key=lambda x: x['path'])
    rng.shuffle(result_list)
    to_remove = result_list[MAX_SEGMENTS_PER_INSTRUMENT:]
    result_list[:] = result_list[:MAX_SEGMENTS_PER_INSTRUMENT]
    
    for result in to_remove:
        os.remove(os.path.join(OUTPUT_DIR, result['path']))
    num_removed += len(to_remove)

print(f'Removed {num_removed} / {num_total} files')

results_old = list(results)
results = [item for result_list in results_by_instrument.values() for item in result_list]
results.sort(key=lambda x: x['path'])

Removed 4129 / 5511 files


In [12]:
with open('metadata_single.json', 'w') as f:
    json.dump(results, f)

In [13]:
voices_bins = np.quantile([r['stats']['num_voices'] for r in results], [0., 0.5, 1.])
pitch_bins = np.quantile([r['stats']['mean_pitch'] for r in results], [0., 0.5, 1.])

def assign_bin(a, bins):
    n = np.digitize(a, bins)
    return np.clip(n, a_min=1, a_max=len(bins) - 1)

binned = collections.defaultdict(list)
for item in results:
    bin_key = 'voices{}_pitch{}'.format(assign_bin(item['stats']['num_voices'], voices_bins),
                                        assign_bin(item['stats']['mean_pitch'], pitch_bins))
    binned[bin_key].append(item)

print({k: len(v) for k, v in binned.items()})

{'voices1_pitch1': 323, 'voices2_pitch1': 368, 'voices1_pitch2': 368, 'voices2_pitch2': 323}


In [14]:
pairs = []
for bin_key in sorted(binned.keys()):
    rng = np.random.default_rng(seed=0)
    metas = list(binned[bin_key])
    rng.shuffle(metas)
    pairs.extend(zip(metas[::2], metas[1::2]))

with open('pairs', 'w') as f:
    for meta1, meta2 in pairs:
        print(os.path.join(OUTPUT_DIR, meta1['path']),
              os.path.join(OUTPUT_DIR, meta2['path']),
              sep='\t', file=f)

print(f'Saved {len(pairs)} pairs')

Saved 690 pairs


In [15]:
!head pairs

wav_16kHz/MERCMusic_Knockout_MERCMusic_Knockout_Full_31_Horns.179.wav	wav_16kHz/AnnaBlanton_Rachel_AnnaBlanton_Rachel_Full_07_Viola.183.wav
wav_16kHz/SixFoxWhiskey_HopeAndTheSea_Six Fox Whiskey_Hope and the Sea_ELE Guitar1-Proto.0450.wav	wav_16kHz/ZaneCarneyAndFriends_TalkToMeBaby_Zane Carney @ Vintage King LA TELEFUNKEN_E GUITAR 3 - M81 - API 1608.0124.wav
wav_16kHz/MR1110_YoungGriffo_MR1110_YoungGriffo_Full_10_BassAmpMic2.0011.wav	wav_16kHz/MR1110_YoungGriffo_MR1110_YoungGriffo_Full_14_ElecGtr1d_OrangeFatHead.064.wav
wav_16kHz/MR1003_Zeno_MR1003_Zeno_Full_33_ElecGtr6DI.081.wav	wav_16kHz/TheBrew_WhatIWant_BREW-REMIX-GUITAR-BASS_BASS CAB - CU-29 copperhead - BAE 1073.0173.wav
wav_16kHz/Fergessen_TheWind_Fergessen_TheWind_Full_21_AcGtr_Mic3.042.wav	wav_16kHz/HurrayForTheRiffRaff_LivingInTheCity_Hurray for the Riff Raff_Livin in the City_Keys-Amp-M81.83.wav
wav_16kHz/AdmiralCrumple_KeepsFlowing_AdmiralCrumple_KeepsFlowing_Full_06_Sample.080.wav	wav_16kHz/TallHeights_SpiritCold_18. Tall H